In [1]:
# from kubernetes_asyncio import client, config
import kubernetes_asyncio as kube
import asyncio

In [2]:
async def list_pods_once():
	await kube.config.load_kube_config()
	api = kube.client.CoreV1Api()
	
	pod_list_response = await api.list_namespaced_pod('cvlab')
	
	print(' '.join(f'{pod.metadata.name}' for pod in pod_list_response.items))
	
	return pod_list_response
	
# asyncio.run(main())
pod_list = await list_pods_once()

# Configs can be set in Configuration class directly or using helper
# utility. If no argument provided, the config will be loaded from
# default location.


bedn bedn2 benlalahpod.com katircio-imagenet katircio-imagenet2 katircio-localiz kicirogl-airsim-2 lis-image-test mv-cvpr3 nakka-advseg11 nakka-advseg14 nakka-advseg20 nsd oner-rt-fulldata-40 oner-rt-fulldata-40-2 rbermude-multiflow remelli-mvp s2t1-ud-l-1 sguo-pytorch-imagenet-iclr weliu-cvpr wickrama-experiments-1 wickrama-experiments-2 wickrama-experiments-3 wickrama-experiments-4 wickrama-experiments-5 wwang-imagenet0 wwang-imagenet1 wwang-imagenet2


In [3]:
pod_to_test = pod_list.items[0]
print(f'Testing on pod [{pod_to_test.metadata.name}]')

Testing on pod [bedn]


### GPU status with nvidia-smi

Example queries: <https://nvidia.custhelp.com/app/answers/detail/a_id/3751/~/useful-nvidia-smi-queries>  
List of fields: <https://briot-jerome.developpez.com/fichiers/blog/nvidia-smi/list.txt>

`nvidia-smi --format=csv --loop=2 --query-gpu=index,utilization.gpu,utilization.memory,memory.used,memory.total`

Injections:  
`kubectl exec -it container -- /usr/bin/nvidia-smi --format=csv --loop=2 --query-gpu=index,utilization.gpu,utilization.memory,memory.used,memory.total`

Run command for 10s:  
`timeout 10 something`  
`timeout 30 nvidia-smi --format=csv --loop=3 --query-gpu=index,utilization.gpu,utilization.memory,memory.used,memory.total`  


In [5]:
api = kube.client.CoreV1Api()
api.connect_get_namespaced_pod_exec?

Signature: api.connect_get_namespaced_pod_exec(name, namespace, **kwargs)
Docstring:
connect_get_namespaced_pod_exec  # noqa: E501

connect GET requests to exec of Pod  # noqa: E501
This method makes a synchronous HTTP request by default. To make an
asynchronous HTTP request, please pass async_req=True
>>> thread = api.connect_get_namespaced_pod_exec(name, namespace, async_req=True)
>>> result = thread.get()

:param async_req bool
:param str name: name of the PodExecOptions (required)
:param str namespace: object name and auth scope, such as for teams and projects (required)
:param str command: Command is the remote command to execute. argv array. Not executed within a shell.
:param str container: Container in which to execute the command. Defaults to only container if there is only one container in the pod.
:param bool stderr: Redirect the standard error stream of the pod for this call. Defaults to true.
:param bool stdin: Redirect the standard input stream of the pod for this call. D

### Executing inside a container

Example: <https://github.com/tomplus/kubernetes_asyncio/blob/master/examples/example3.py>

In [4]:
GPU_QUERY_FIELDS = [
	'index',
	'utilization.gpu',
	'utilization.memory',
	'memory.used',
	'memory.total',
]
GPU_QUERY_CMD = [
	'/usr/bin/timeout', '5',
	'/usr/bin/nvidia-smi',
	'--format=csv',
  	'--loop=1',
	f'--query-gpu={",".join(GPU_QUERY_FIELDS)}',
]
print(GPU_QUERY_CMD)

async def query_gpu_status():
	await kube.config.load_kube_config()
	api_ws = kube.client.CoreV1Api(api_client=kube.stream.WsApiClient())
# 	api_ws = kube.client.CoreV1Api()

	cmd = GPU_QUERY_CMD
	name = 'sguo-pytorch-imagenet-iclr'
# 	name = 'wwang-imagenet2'
	namespace = 'cvlab'
	
# 	cmd = ['ls']
	
	req = api_ws.connect_get_namespaced_pod_exec(
		name = name, 
		namespace = namespace,
		command = cmd,
		stderr=True,
		stdin=False,
		stdout=True,
		tty=False,

# 		async_req=True,
# 		_preload_content=False,
	)
	
	print('Req', req)
	response = await req
	print("Query response: ", response)
	return response
	
# 	w = kube.watch.Watch()
	
# 	async for event in w.stream(api_ws.connect_get_namespaced_pod_exec,
# 		name = name, 
# 		namespace = namespace,
# 		command = cmd,
# 		stderr=True,
# 		stdin=False,
# 		stdout=True,
# 		tty=False,
								
# 		async_req=True,
# 		_preload_content=False,
# 	):
# 		print(event)
	

# 	return response

r = await query_gpu_status()

['/usr/bin/timeout', '5', '/usr/bin/nvidia-smi', '--format=csv', '--loop=1', '--query-gpu=index,utilization.gpu,utilization.memory,memory.used,memory.total']
Req <coroutine object ApiClient.__call_api at 0x7fb36727a5f0>
Query response:  index, utilization.gpu [%], utilization.memory [%], memory.used [MiB], memory.total [MiB]
0, 0 %, 0 %, 8932 MiB, 32510 MiB
1, 0 %, 0 %, 8924 MiB, 32510 MiB
0, 87 %, 18 %, 8932 MiB, 32510 MiB
1, 31 %, 1 %, 8924 MiB, 32510 MiB
0, 95 %, 33 %, 8932 MiB, 32510 MiB
1, 84 %, 27 %, 8924 MiB, 32510 MiB
0, 88 %, 29 %, 8932 MiB, 32510 MiB
1, 41 %, 23 %, 8924 MiB, 32510 MiB
0, 0 %, 0 %, 8932 MiB, 32510 MiB
1, 0 %, 0 %, 8924 MiB, 32510 MiB



In [48]:

def process_row_id(val):
	return int(val)

def process_row_percent(val):
	return float(val.split(maxsplit=1)[0]) * 0.01

def process_row_mem(val):
	return float(val.split(maxsplit=1)[0])

GPU_QUERY_PROCESSORS = {
	'index': int,
	'utilization.gpu': process_row_percent,
	'utilization.memory': process_row_percent,
	'memory.used': process_row_mem,
	'memory.total': process_row_mem,
}


def process_nvidiasmi_line(report_line):
	try:
		return {
			field: GPU_QUERY_PROCESSORS[field](value.strip())
			for field, value in zip(GPU_QUERY_FIELDS, report_line.split(','))
		}
	except Exception as e:
		print(f'{e} in [{report_line}]')

def process_nvidiasmi_report(report_str):
	
	report_lines = report_str.split('\n')
	
	report_data = [
		process_nvidiasmi_line(line) for line in report_lines[1:]
	]
	
	return report_data
	
process_nvidiasmi_report(r)
	

invalid literal for int() with base 10: '' in []


[{'index': 0,
  'utilization.gpu': 0.0,
  'utilization.memory': 0.0,
  'memory.used': 8932.0,
  'memory.total': 32510.0},
 {'index': 1,
  'utilization.gpu': 0.0,
  'utilization.memory': 0.0,
  'memory.used': 8924.0,
  'memory.total': 32510.0},
 {'index': 0,
  'utilization.gpu': 0.87,
  'utilization.memory': 0.18,
  'memory.used': 8932.0,
  'memory.total': 32510.0},
 {'index': 1,
  'utilization.gpu': 0.31,
  'utilization.memory': 0.01,
  'memory.used': 8924.0,
  'memory.total': 32510.0},
 {'index': 0,
  'utilization.gpu': 0.9500000000000001,
  'utilization.memory': 0.33,
  'memory.used': 8932.0,
  'memory.total': 32510.0},
 {'index': 1,
  'utilization.gpu': 0.84,
  'utilization.memory': 0.27,
  'memory.used': 8924.0,
  'memory.total': 32510.0},
 {'index': 0,
  'utilization.gpu': 0.88,
  'utilization.memory': 0.29,
  'memory.used': 8932.0,
  'memory.total': 32510.0},
 {'index': 1,
  'utilization.gpu': 0.41000000000000003,
  'utilization.memory': 0.23,
  'memory.used': 8924.0,
  'memory.t

In [27]:
import numpy as np
from io import StringIO
np.loadtxt(StringIO(r), skiprows=1, delimiter=',', names=GPU_QUERY_FIELDS, converters={k.replace('.', ''): v for k, v in GPU_QUERY_PROCESSORS.items()})

TypeError: loadtxt() got an unexpected keyword argument 'names'

In [51]:
res_parsed = np.genfromtxt(
	StringIO(r), 
	delimiter = ',', 
	autostrip = True,
	dtype=None, # precent casting to float
 	names=GPU_QUERY_FIELDS,
 	skip_header = 1, 
# 	names=True,
	deletechars = '', # prevent mangling of names
	converters = GPU_QUERY_PROCESSORS,
)

In [52]:
res_parsed

array([(0, 0.  , 0.  , 8932., 32510.), (1, 0.  , 0.  , 8924., 32510.),
       (0, 0.87, 0.18, 8932., 32510.), (1, 0.31, 0.01, 8924., 32510.),
       (0, 0.95, 0.33, 8932., 32510.), (1, 0.84, 0.27, 8924., 32510.),
       (0, 0.88, 0.29, 8932., 32510.), (1, 0.41, 0.23, 8924., 32510.),
       (0, 0.  , 0.  , 8932., 32510.), (1, 0.  , 0.  , 8924., 32510.)],
      dtype=[('index', '<i4'), ('utilization.gpu', '<f8'), ('utilization.memory', '<f8'), ('memory.used', '<f8'), ('memory.total', '<f8')])

In [55]:
mem_relative = res_parsed['memory.used'] / res_parsed['memory.total']
mem_relative

array([0.27474623, 0.27450015, 0.27474623, 0.27450015, 0.27474623,
       0.27450015, 0.27474623, 0.27450015, 0.27474623, 0.27450015])